Imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import warnings
warnings.simplefilter(action='ignore')
from tqdm.notebook import tqdm
import pickle
import json

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ngeva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ngeva\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


read csv

In [6]:
df_raw = pd.read_csv('data.csv')

In [7]:
df = df_raw.loc[:,['text', 'user_followers', 'favorites', 'retweets', 'is_retweet', 'category']]

In [8]:
df = df.drop_duplicates()

Pre Processing text column

In [9]:
df["up_text"] = df.text.str.lower()
df.up_text = df.up_text.apply(lambda x:re.sub('@[^\s]+','',x))
df.up_text = df.up_text.apply(lambda x:re.sub(r'\B#\S+','',x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"http\S+", "", x))
df.up_text = df.up_text.apply(lambda x:' '.join(re.findall(r'\w+', x)))
df.up_text = df.up_text.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
df.up_text = df.up_text.apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

In [10]:
contractions_dict = { "ain't": "are not ","'s":" is ","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

df['up_text']=df['up_text'].apply(lambda x:expand_contractions(x))

creating tokenized column

In [11]:
df['tokenized'] = df['up_text'].apply(word_tokenize)
stop_words = set(stopwords.words('english'))
df['tokenized'] = df['tokenized'].apply(lambda x: [word for word in x if word not in stop_words])
df['pos_tags'] = df['tokenized'].apply(nltk.tag.pos_tag)

In [12]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
df['tokenized'] = df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

In [13]:
wnl = WordNetLemmatizer()
df['tokenized'] = df['tokenized'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])

new feature - number of words in tweet

In [14]:
df['n_words'] = df['up_text'].apply(lambda x: len(x.split()))

split to tarin and test

In [15]:
features = list(df.columns)
features.remove('category')

In [16]:
from sklearn.model_selection import train_test_split
X = df[features]
y = df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, stratify=df['category'])

finding the most common words from train to create features

In [17]:
words = X_train['tokenized']
allwords = []
for wordlist in words:
    allwords += wordlist
allwords = [word for word in allwords if word !='amp']
allwords = [word for word in allwords if word !='rt']

In [18]:
mostcommon = FreqDist(allwords).most_common(100)
mostcommon = [tup[0] for tup in mostcommon]

In [19]:
def most_common_token(token):
    new_token=[]
    for word in token:
        if word in mostcommon:
            new_token.append(word)
    return new_token

In [20]:
X_train['tokenized_common'] = X_train['tokenized'].apply(lambda x: most_common_token(x))
X_test['tokenized_common'] = X_test['tokenized'].apply(lambda x: most_common_token(x))

In [21]:
empty_tokens = X_train[(X_train['tokenized_common'].str.len() == 0)].index
X_train = X_train.drop(empty_tokens)
y_train = y_train.drop(empty_tokens)

empty_tokens = X_test[(X_test['tokenized_common'].str.len() == 0)].index
X_test = X_test.drop(empty_tokens)
y_test = y_test.drop(empty_tokens)

In [22]:
for word in mostcommon:
    X_train[word]=0
    X_test[word]=0

In [23]:
X_train = X_train.reset_index()
X_test = X_test.reset_index()

In [24]:
for index, row in tqdm(enumerate(X_train.tokenized_common)):
    for word in row:
        X_train.loc[index,word] = 1
        
for index, row in tqdm(enumerate(X_test.tokenized_common)):
    for word in row:
        X_test.loc[index,word] = 1

In [25]:
y_train = y_train.replace({'Positive':0, 'Negative':1})
X_train['is_retweet'] = X_train['is_retweet'].replace({False:0, True:1})

y_test = y_test.replace({'Positive':0, 'Negative':1})
X_test['is_retweet'] = X_test['is_retweet'].replace({False:0, True:1})

In [26]:
columns_to_drop = ['text', 'up_text', 'pos_tags','tokenized','index','tokenized_common']
X_train = X_train.drop(columns_to_drop, axis=1)
X_test = X_test.drop(columns_to_drop, axis=1)

In [27]:
X_train

,user_followers,favorites,retweets,is_retweet,n_words,vaccine,get,covid,make,today,...,like,want,use,love,well,old,moment,last,look,roll
0,491,0,7,1,20,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1525,0,100,1,18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6333,0,1,1,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,755,0,5,1,21,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,246,0,0,0,13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12485,273,0,14,1,17,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12486,602,0,12,1,20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12487,3867,0,28,1,18,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12488,194,0,95,1,14,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


model

In [28]:
import xgboost as xgb

In [29]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

xgb = XGBClassifier()

folds = 3
param_comb = 5
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

parameters = {
     "eta"              : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"        : [ 3, 4, 5, 6, 8, 10],
     "min_child_weight" : [ 1, 2, 3, 5 ],
     "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     "colsample_bytree" : [0.5 , 0.7, 1]
     }


random_search = RandomizedSearchCV(xgb, param_distributions=parameters, 
                                   n_iter=param_comb, 
                                   scoring='roc_auc',
                                   n_jobs=4, 
                                   cv=skf.split(X_train,y_train), 
                                   verbose=3, 
                                   random_state=1001 )

random_search.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[15:32:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x0000012D82DB8BA0>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, mis...
                                           random_state=None, reg_alpha=None,
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=None, tr

In [30]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)


 All results:
{'mean_fit_time': array([72.38200037, 52.12933644, 19.88309765, 26.58902979, 22.39267802]), 'std_fit_time': array([ 0.44325366, 21.62757531,  3.34570145,  0.57673053,  4.62822354]), 'mean_score_time': array([0.16567151, 0.17533151, 0.17936095, 0.11467163, 0.16521605]), 'std_score_time': array([0.00612348, 0.03583269, 0.06525963, 0.01725433, 0.07685742]), 'param_min_child_weight': masked_array(data=[2, 3, 5, 1, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[6, 10, 5, 4, 5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[0.0, 0.1, 0.4, 0.3, 0.4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_eta': masked_array(data=[0.25, 0.15, 0.1, 0.25, 0.25],
             mask=[False, False, False, False, False],
       fill_value='?

saving model for deployment

In [31]:
model = random_search.best_estimator_

In [36]:
import pickle

with open('xgboost.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=4)

In [35]:
filename = "xgboost.sav"
pickle.dumps(model, open(filename, 'wb'))

TypeError: an integer is required (got type _io.BufferedWriter)

results and metrics

In [37]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, roc_auc_score, classification_report
preds = random_search.predict_proba(X_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

Precision = 0.9206919060052219
Recall = 0.8905764714794333
Accuracy = 0.9226075786769429


In [ ]:
prec = round(precision_score(y_test, best_preds, average='macro'),2)
recall = round(recall_score(y_test, best_preds, average='macro'),2)
acc = round(accuracy_score(y_test, best_preds),2)

In [ ]:
tn, fp, fn, tp = confusion_matrix(np.array(y_test), best_preds).ravel()
fpr = fp/(fp+tn)
tpr = tp/(tp+fn)
auc = roc_auc_score(y_test, preds[:,1])

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, preds[:,1])
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='xgboost')
display.plot() 
plt.plot([0,1],[0,1], ls= '--', label = 'ROC curve (area = %0.2f)' % auc)
plt.title('XGboost classifier\n accuray={}, precision={}, recall={}'.format(acc,prec,recall))
plt.show()      

In [ ]:
target_names = ['non-anti-vax', 'anti-vax']
print(classification_report(y_test, best_preds, target_names=target_names))